## List of all 

### Testing with one country

In [1]:
import requests
import pandas as pd
resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc=MM&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=category_code')
j = resp.json()
df = pd.json_normalize(j, record_path='result')

In [2]:
print(df.head())

   anomaly_count category_code  confirmed_count  failure_count  \
0             13          ALDR                0              0   
1             23          ANON                3              2   
2              2          COMM                0              0   
3              6          COMT                0              1   
4              2          CTRL                0              0   

   measurement_count measurement_start_day  ok_count  
0                 93            2022-07-01        80  
1                204            2022-07-01       176  
2                 36            2022-07-01        34  
3                128            2022-07-01       121  
4                  5            2022-07-01         3  


### List of all inputs of all countries

In [3]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam, Timor Leste
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN',
    'TL'
]
def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=category_code')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    return df

# Note: as_index=False is to combine the first 2 headers when doing agg

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_all = pd.concat(df_list)

In [5]:
df_all.to_csv('webconnectivity_2023.csv')

In [4]:
print(df_all.head())

   anomaly_count category_code  confirmed_count  failure_count  \
0             13          ALDR                0              0   
1             23          ANON                3              2   
2              2          COMM                0              0   
3              6          COMT                0              1   
4              2          CTRL                0              0   

   measurement_count measurement_start_day  ok_count probe_cc  
0                 93            2022-07-01        80       MM  
1                204            2022-07-01       176       MM  
2                 36            2022-07-01        34       MM  
3                128            2022-07-01       121       MM  
4                  5            2022-07-01         3       MM  


### aggregation

In [6]:
import requests
import pandas as pd
resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc=MM&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=input')
j = resp.json()
df_date_input = pd.json_normalize(j, record_path='result')

In [11]:
df_date_input['measurement_start_day'] = pd.to_datetime(df_date_input['measurement_start_day'])


In [13]:
df_date_input['quarter'] = df_date_input['measurement_start_day'].dt.to_period('Q')

In [ ]:
print(df_date_input.head())

In [20]:
df_input_summary = df_date_input.groupby(['quarter'], as_index=False).agg(
        input_count = ('input', 'nunique')
        )

In [21]:
print(df_input_summary.head())

  quarter  input_count
0  2022Q3         2570
1  2022Q4         2440
2  2023Q1         2449
3  2023Q2         2485


In [25]:
import requests
import pandas as pd
resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc=MM&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=probe_asn')
j = resp.json()
df_date_asn = pd.json_normalize(j, record_path='result')
df_date_asn['measurement_start_day'] = pd.to_datetime(df_date_asn['measurement_start_day'])
df_date_asn['quarter'] = df_date_asn['measurement_start_day'].dt.to_period('Q')
df_date_asn['probe_cc'] = 'MM'

In [26]:
df_asn_summary = df_date_asn.groupby(['quarter','probe_cc'], as_index=False).agg(
        asn_count = ('probe_asn', 'nunique')
        )

In [27]:
print(df_asn_summary.head())

  quarter probe_cc  asn_count
0  2022Q3       MM         32
1  2022Q4       MM         24
2  2023Q1       MM         27
3  2023Q2       MM         28


In [22]:
df_quarter = df_asn_summary.merge(df_input_summary, on='quarter', how='inner')


In [23]:
print(df_quarter.head())

  quarter  asn_count  input_count
0  2022Q3         32         2570
1  2022Q4         24         2440
2  2023Q1         27         2449
3  2023Q2         28         2485


In [5]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam, Timor Leste
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN',
    'TL'
]
def get_webconnectivity(probe_cc):
    resp_asn = requests.get('https://api.ooni.io/api/v1/aggregation'
                     f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=probe_asn')
    j = resp_asn.json()
                            
    df_date_asn = pd.json_normalize(j, record_path='result')
    df_date_asn['measurement_start_day'] = pd.to_datetime(df_date_asn['measurement_start_day'])
    df_date_asn['quarter'] = df_date_asn['measurement_start_day'].dt.to_period('Q')
    df_date_asn['probe_cc'] = probe_cc
    
    df_asn_summary = df_date_asn.groupby(['quarter','probe_cc'], as_index=False).agg(
        asn_count = ('probe_asn', 'nunique'))    
                            
    resp_input = requests.get('https://api.ooni.io/api/v1/aggregation'
                        f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=input')
    k = resp_input.json()
                              
    df_date_input = pd.json_normalize(k, record_path='result')
    df_date_input['measurement_start_day'] = pd.to_datetime(df_date_input['measurement_start_day'])
    df_date_input['quarter'] = df_date_input['measurement_start_day'].dt.to_period('Q')
    df_date_input['probe_cc'] = probe_cc
                              
    df_input_summary = df_date_input.groupby(['quarter'], as_index=False).agg(
        input_count = ('input', 'nunique'),
        ok_count = ('ok_count', 'sum'),
        anomaly_count = ('anomaly_count', 'sum'),
        confirmed_count = ('confirmed_count', 'sum'),
        failure_count = ('failure_count', 'sum'),
        measurement_count = ('measurement_count', 'sum')
    )                     
    
    df_quarter = df_asn_summary.merge(df_input_summary, on='quarter', how='inner')

    return df_quarter

# Note: as_index=False is to combine the first 2 headers when doing agg

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_all_2 = pd.concat(df_list)

In [ ]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam, Timor Leste
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN',
    'TL'
]
def get_webconnectivity(probe_cc):
    resp_asn = requests.get('https://api.ooni.io/api/v1/aggregation'
                     f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=probe_asn')
    j = resp_asn.json()
                            
    df_date_asn = pd.json_normalize(j, record_path='result')
    df_date_asn['measurement_start_day'] = pd.to_datetime(df_date_asn['measurement_start_day'])
    df_date_asn['quarter'] = df_date_asn['measurement_start_day'].dt.to_period('Q')
    df_date_asn['probe_cc'] = probe_cc
    
    df_asn_summary = df_date_asn.groupby(['quarter','probe_cc'], as_index=False).agg(
        asn_count = ('probe_asn', 'nunique'))    
                            
    resp_input = requests.get('https://api.ooni.io/api/v1/aggregation'
                        f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=input')
    k = resp_input.json()
                              
    df_date_input = pd.json_normalize(k, record_path='result')
    df_date_input['measurement_start_day'] = pd.to_datetime(df_date_input['measurement_start_day'])
    df_date_input['quarter'] = df_date_input['measurement_start_day'].dt.to_period('Q')
    df_date_input['probe_cc'] = probe_cc
                              
    df_input_summary = df_date_input.groupby(['quarter'], as_index=False).agg(
        input_count = ('input', 'nunique'),
        ok_count = ('ok_count', 'sum'),
        anomaly_count = ('anomaly_count', 'sum'),
        confirmed_count = ('confirmed_count', 'sum'),
        failure_count = ('failure_count', 'sum'),
        measurement_count = ('measurement_count', 'sum')
    )                     
    
    df_quarter = df_asn_summary.merge(df_input_summary, on='quarter', how='inner')

    return df_asn_summary

# Note: as_index=False is to combine the first 2 headers when doing agg

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_all_ = pd.concat(df_list)

In [32]:
print(df_all_2.head())

  quarter probe_cc_x  asn_count probe_cc_y  input_count
0  2022Q3         MM         32         MM         2570
1  2022Q4         MM         24         MM         2440
2  2023Q1         MM         27         MM         2449
3  2023Q2         MM         28         MM         2485
0  2022Q3         KH         14         KH         1886


In [ ]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam, Timor Leste
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN',
    'TL'
]
def get_webconnectivity(probe_cc):
    resp_asn = requests.get('https://api.ooni.io/api/v1/aggregation'
                     f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=probe_asn')
    j = resp_asn.json()
                            
    df_date_asn = pd.json_normalize(j, record_path='result')
    df_date_asn['measurement_start_day'] = pd.to_datetime(df_date_asn['measurement_start_day'])
    df_date_asn['quarter'] = df_date_asn['measurement_start_day'].dt.to_period('Q')
    df_date_asn['probe_cc'] = probe_cc
    
    df_asn_summary = df_date_asn.groupby(['quarter','probe_cc'], as_index=False).agg(
        asn_count = ('probe_asn', 'nunique'))    
                            
    resp_input = requests.get('https://api.ooni.io/api/v1/aggregation'
                        f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2022-07-01&until=2023-07-01&axis_x=measurement_start_day&axis_y=input')
    k = resp_input.json()
                              
    df_date_input = pd.json_normalize(k, record_path='result')
    df_date_input['measurement_start_day'] = pd.to_datetime(df_date_input['measurement_start_day'])
    df_date_input['quarter'] = df_date_input['measurement_start_day'].dt.to_period('Q')
    df_date_input['probe_cc'] = probe_cc
                              
    df_input_summary = df_date_input.groupby(['quarter'], as_index=False).agg(
        input_count = ('input', 'nunique'),
        ok_count = ('ok_count', 'sum'),
        anomaly_count = ('anomaly_count', 'sum'),
        confirmed_count = ('confirmed_count', 'sum'),
        failure_count = ('failure_count', 'sum'),
        measurement_count = ('measurement_count', 'sum')
    )                     
    
    df_quarter = df_asn_summary.merge(df_input_summary, on='quarter', how='inner')

    return df_quarter

# Note: as_index=False is to combine the first 2 headers when doing agg

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_all_2 = pd.concat(df_list)

In [6]:
df_all_2.to_csv('webconnectivity_2023_2.csv')